In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import sys
sys.path.append('..')
from PIL import Image
from tqdm import trange
import gymnasium as gym
import numpy as np
from sapien.core import Pose
import matplotlib.pyplot as plt
import os
import warnings
import shutil
import random
from plot_utils import *
from itertools import product, combinations, permutations

from utils import *
from agent_env_utils import *
from config import color_maps, background_quadrants
from scene_helpers import get_random_position, create_config, create_directories, finalize_quadrant


In [ ]:
from agent_env_utils import *
import math
data_dir = "benchmark/direction"

objects = ["cube","sphere","cube","sphere",'cube',"cube"]
obj_num = len(objects)

flush = True
if flush:
    if os.path.exists(data_dir):
        shutil.rmtree(data_dir)
        os.makedirs(data_dir)

# initialize pair of trajectories (The last one is random)
moving_directions = [["front"],["behind"], ["left"],["right"],["left_behind"],["left_front"],["right_front"],["right_behind"]]
# initialize pair of colors
strings = color_maps.keys()
pairs = list(itertools.permutations(strings, obj_num))
random.seed(42)
random.shuffle(pairs)
colors = pairs[0]
potential_sizes = [0.03, 0.035]
direction_length  = 0.2
scales = [1.1,1.4]
ranges =[(-0.4, 0.4), (-0.4, 0.3)] #[(0.0, 0.02), (-0.2, -0.19)]
configs = []
speeds = [5,10]



import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    env = gym.make("CustomEnv-v0", obs_mode="rgbd")

    outputs = []
    # loop through two speed
    for obj_index, obj_type in enumerate(objects):                                           
        #cube1_config = {'size': np.random.uniform(0.05, 0.08, 1)[0], 'color': color_maps[color_1], 'obj_type': "cube", 'ranges': [(-0.3, 0.3), (-0.3, 0.25)]}
        #cube2_config = {'size': np.random.uniform(0.05, 0.08, 1)[0], 'color': color_maps[color_2], 'obj_type': "cube", 'ranges': [(-0.3, 0.3), (-0.3, 0.25)]}
        size = random.choice(potential_sizes)
        color = colors[obj_index]
        ranges = [(-0.2, 0.2), (-0.15, 0.15)] if obj_index == 1 else [(-0.4, 0.4), (-0.4, 0.3)]
        if(obj_type == 'custom'):
            obj_config = {'obj_type': obj_type, 'name': "002_master_chef_can", 'size': size*1.1/2, 'ranges': ranges, 'static': True}
        else:
            obj_config = {'size': size, 'color': color_maps[color], 'obj_type': obj_type, 'ranges': ranges, 'static': True}
            #random_ball_configs = [{'size': np.random.uniform(0.01, 0.05, 1)[0], 'color': color_maps["purple"], 'obj_type': "sphere", 'ranges': [(-0.5, 0.3), (-0.5, 0.4)]} for _ in range(5)]

        configs += [obj_config] #+ random_ball_configs
        #configs = [custom_config]
            
    for k,directions in enumerate(moving_directions):
        direction = directions[0]
        #get the obj's initialize position without blocking the path, get moving obj and ref obj position
        env, initilization_poses, mov_pos = initialize_obj_nooverlap_path(direction, configs, direction_length, env)
        initilization_poses[0] = mov_pos 
        env.set_poses(initilization_poses)
        direction_configs = [(0, 0, direction) for direction in directions]
        distances = [direction_length for _ in directions]
        frames = env.move_in_directions(direction_configs, distances, steps=None, camera_view="all")
        video_name = f"traj{k}_{objects[0]}_{colors[0]}"
        video_path = os.path.join(data_dir, f"videos/{video_name}.mp4")
        os.makedirs(os.path.dirname(video_path), exist_ok=True)
        generate_video(video_path, frames['top0'], fps=10, color_change=True)


        image_dir = os.path.join(data_dir, f"traj{k}_move_{colors[0]}{objects[0]}_ref_{colors[1]}{objects[1]}")

        save_images_pararell(frames['top0'], image_dir)
                    
    write_out(f"{data_dir}/index.jsonl", outputs)
            
                

del env